Referred doc-https://huggingface.co/docs/transformers/tasks/sequence_classification


- Show the runtime used (T4)
- Upload the financial_sentiment_data.csv file to Colab

In [42]:
# pip install numpy==1.26.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 123.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.4.0 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.4.0 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.4.0 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.4.0 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.4.0 which is incompatible.


In [1]:
pip install evaluate

In [2]:
import numpy as np
print(np.__version__)

2.4.0


Importing required libraries

In [3]:
import numpy as np
import pandas as pd

https://www.kaggle.com/datasets/sbhatti/financial-sentiment-analysis?select=data.csv

In [4]:
financial_data = pd.read_csv("/content/financial_sentiment_data.csv")

financial_data.sample(10)

,Sentence,Sentiment
2223,These six agreements are the company 's first ...,neutral
557,All of Raisio 's divisions recorded an operati...,positive
1417,Compass Group says positive for year ahead,positive
3378,The stock rose for a third day on Tuesday brin...,positive
3112,Teleste expects to start the deliveries at the...,neutral
318,The corrected chapter is in its entirety below .,neutral
622,Net sales have been eaten by the weak US dollar .,negative
2536,"SAN FRANCISCO ( MarketWatch ) -- Nokia Corp , ...",neutral
5736,Altogether CapMan employs approximately 150 pe...,neutral
3817,Danske Bank A-S DANSKE DC jumped 3.7 percent t...,positive


Checking dimension of data

In [5]:
financial_data.shape

(5842, 2)

In [6]:
financial_data["Sentiment"].value_counts()

,count
Sentiment,
neutral,3130
positive,1852
negative,860


In [7]:
pd.set_option('future.no_silent_downcasting', True)

financial_data["label"] = financial_data["Sentiment"].replace({"negative": 0, "neutral": 1, "positive": 2}).astype('int64')

financial_data.sample(10)

,Sentence,Sentiment,label
3743,The company 's transportation business is cond...,neutral,1
2915,LONDON MarketWatch -- Share prices ended lower...,negative,0
1347,$FTI Breakout today after reporting earnings l...,positive,2
4233,The second company acquired is Sweden 's Refte...,neutral,1
4789,Okmetic 's silicon wafers are part of a furthe...,neutral,1
1780,The value of the order is about EUR 30mn .,neutral,1
4214,The other actions include the cutting of the e...,neutral,1
2562,Travis Perkins Hikes Dividend 20% As Profit An...,positive,2
570,Operating profit decreased to EUR 11.2 mn from...,negative,0
353,The new majority owners of Aspocomp Thailand C...,neutral,1


Converting pandas df to huggingface dataset

In [8]:
from datasets import Dataset

financial_ds = Dataset.from_pandas(financial_data)

financial_ds

Dataset({
    features: ['Sentence', 'Sentiment', 'label'],
    num_rows: 5842
})

Casts the given column as :obj:datasets.features.ClassLabel and updates the table.

In [9]:
financial_ds = financial_ds.class_encode_column("label")

Stringifying the column:   0%|          | 0/5842 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/5842 [00:00<?, ? examples/s]

Splitting the dataset’s  into a train and test set with the train_test_split method

In [10]:
financial_ds = financial_ds.train_test_split(test_size = 0.2, stratify_by_column = 'label', seed = 123)

financial_ds

DatasetDict({
    train: Dataset({
        features: ['Sentence', 'Sentiment', 'label'],
        num_rows: 4673
    })
    test: Dataset({
        features: ['Sentence', 'Sentiment', 'label'],
        num_rows: 1169
    })
})

Negative, positive , neutral instances can be seen

In [11]:
financial_ds['train'][1]

{'Sentence': '$YHOO A breakout above $29.83 would constitute a technical entry for the short term trader. http://stks.co/jkUF',
 'Sentiment': 'positive',
 'label': 2}

In [12]:
financial_ds['train'][2]

{'Sentence': 'New Chairman of the Board of Directors , Mr Chaim Katzman , will give a presentation and answer questions .',
 'Sentiment': 'neutral',
 'label': 1}

In [13]:
financial_ds['train'][26]

{'Sentence': '@chessNwine: $IWM 30-Minute Chart. Small caps threatening descending triangle breakdown under $110.20.  http://stks.co/r0KKm',
 'Sentiment': 'negative',
 'label': 0}

https://huggingface.co/roberta-base

NOTE that this is a model pre-trained using masked language modeling (MLM) and we are going to fine-tune it on a classification task

Here we will set up a token to access the Hugging Face hub

In [14]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Creating a preprocessing function to tokenize text and truncate sequences to be no longer than roberta-base maximum input length (512 tokens)

In [15]:
def preprocess_function(examples):
    return tokenizer(examples["Sentence"], truncation = True)

In [16]:
tokenizer.vocab_size

50265

To apply the preprocessing function over the entire dataset, use 🤗 Datasets map function. You can speed up map by setting batched=True to process multiple elements of the dataset at once:

In [17]:
tokenized_financial_ds = financial_ds.map(preprocess_function, batched = True)

Map:   0%|          | 0/4673 [00:00<?, ? examples/s]

Map:   0%|          | 0/1169 [00:00<?, ? examples/s]

In [18]:
financial_ds["train"][45]

{'Sentence': '( ADP News ) - Feb 4 , 2009 - Finnish broadband data communication systems and solutions company Teleste Oyj ( HEL : TLT1V ) said today its net profit decreased to EUR 5.5 million ( USD 7.2 m ) for 2008 from EUR 9.4 million for 200',
 'Sentiment': 'negative',
 'label': 0}

Tokenizing that instance

In [19]:
tokenized_financial_ds["train"][45]

{'Sentence': '( ADP News ) - Feb 4 , 2009 - Finnish broadband data communication systems and solutions company Teleste Oyj ( HEL : TLT1V ) said today its net profit decreased to EUR 5.5 million ( USD 7.2 m ) for 2008 from EUR 9.4 million for 200',
 'Sentiment': 'negative',
 'label': 0,
 'input_ids': [0,
  1640,
  4516,
  510,
  491,
  4839,
  111,
  1927,
  204,
  2156,
  2338,
  111,
  21533,
  11451,
  414,
  4358,
  1743,
  8,
  2643,
  138,
  5477,
  13967,
  17311,
  267,
  36,
  39509,
  4832,
  27017,
  565,
  134,
  846,
  4839,
  26,
  452,
  63,
  1161,
  1963,
  8065,
  7,
  10353,
  195,
  4,
  245,
  153,
  36,
  6775,
  262,
  4,
  176,
  475,
  4839,
  13,
  2266,
  31,
  10353,
  361,
  4,
  306,
  153,
  13,
  1878,
  2],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1

Detokenizing the text to get back the input text

In [20]:
tokenizer.decode(tokenized_financial_ds["train"][45]["input_ids"])

'<s>( ADP News ) - Feb 4 , 2009 - Finnish broadband data communication systems and solutions company Teleste Oyj ( HEL : TLT1V ) said today its net profit decreased to EUR 5.5 million ( USD 7.2 m ) for 2008 from EUR 9.4 million for 200</s>'

Now Creating  a batch of examples using DataCollatorWithPadding. It’s more efficient to dynamically pad the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In NLP, when processing sequences of text (like sentences or paragraphs), it's common to batch these sequences together for efficient training. However, since these sequences can vary in length, DataCollatorWithPadding automatically pads all sequences in a batch to match the length of the longest sequence in that batch. This ensures that all sequences in a batch have the same length, which is a requirement for most deep learning models.

In [21]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

Including a metric during training is often helpful for evaluating your model’s performance. We can quickly load a evaluation method with the 🤗 Evaluate library. For this task, loading the accuracy metric.

In [22]:
import evaluate

accuracy = evaluate.load("accuracy")

Then creating a function that passes your predictions and labels to compute to calculate the accuracy

In [23]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    return accuracy.compute(predictions = predictions, references = labels)

Before you start training your model, creating a map of the expected ids to their labels with id2label and label2id.

In [24]:
id2label = {0: "negative", 1: "neutral", 2: "positive"}

label2id = {"negative": 0, "neutral": 1, "positive": 2}

For Finetuning a model in TensorFlow, first we start by setting up an optimizer function, learning rate schedule, and some training hyperparameters.

create_optimizer creates an AdamWeightDecay optimizer

In [25]:
from transformers import create_optimizer

batch_size = 16
num_epochs = 2

batches_per_epoch = len(tokenized_financial_ds["train"]) // batch_size

total_train_steps = int(batches_per_epoch * num_epochs)

optimizer, schedule = create_optimizer(init_lr = 2e-6, num_warmup_steps = 0, num_train_steps = total_train_steps)

In [26]:
from transformers import AutoModelForSequenceClassification
import torch

model = AutoModelForSequenceClassification.from_pretrained(
    "roberta-base", num_labels = 3, id2label = id2label, label2id = label2id
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Converting train and validation datasets to the tf.data.Dataset format with prepare_tf_dataset():

Configuring the model for training with compile. Note that Transformers models all have a default task-relevant loss function, so you don’t need to specify one unless you want to

In [27]:
from transformers import TrainingArguments
from transformers import Trainer

training_args = TrainingArguments(
    output_dir="financial_sentiment_analysis",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none", # Otherwise will report to W&B
    push_to_hub=False,
)

In [28]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_financial_ds["train"],
    eval_dataset=tokenized_financial_ds["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Write permission is validated by token
- After running the notebook cell - go to https://huggingface.co/
- Go to the top-right corner click on account avatar -> Settings
- Go to Access Tokens - create a Write token
- Copy and paste that in here



In [29]:
from huggingface_hub import notebook_login

notebook_login()

Train the model

In [30]:
train_result = trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.387204,0.802395
2,0.472400,0.383234,0.813516


Starting training our model,Calling  fit with your training and validation datasets, the number of epochs, and your callbacks to finetune the model:

https://huggingface.co/jinxxx123

In [31]:
trainer.push_to_hub("financial_text_sentiment_classification_model")

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...nalysis/model.safetensors:   0%|          | 1.06MB /  499MB            

  ...nalysis/training_args.bin:   3%|3         |   188B / 5.84kB            

CommitInfo(commit_url='https://huggingface.co/jinxxx123/financial_sentiment_analysis/commit/405a5bcdf0b366cdebe8afcb477dfa235ba42eb1', commit_message='financial_text_sentiment_classification_model', commit_description='', oid='405a5bcdf0b366cdebe8afcb477dfa235ba42eb1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/jinxxx123/financial_sentiment_analysis', endpoint='https://huggingface.co', repo_type='model', repo_id='jinxxx123/financial_sentiment_analysis'), pr_revision=None, pr_num=None)


- Go back to the model and show that the model has been pushed (make sure you refresh)
- Show the  model card, files and versions (should NOT be empty), community, and settings

In [32]:
from transformers import pipeline

text = """
  The stock market experienced significant volatility today, with the Dow Jones Industrial Average
  dropping over 500 points in early trading before recovering slightly by the close of the session.
"""

classifier = pipeline("sentiment-analysis", model = model, tokenizer = tokenizer)

classifier(text)

Device set to use cuda:0


[{'label': 'negative', 'score': 0.6128352880477905}]

In [33]:
text = """
  The central bank announced a cut in interest rates, aiming to stimulate economic
  growth by making borrowing cheaper for both consumers and businesses.
"""

classifier(text)

[{'label': 'positive', 'score': 0.9717020988464355}]

In [34]:
text = """
  Inflation rates have been steadily increasing, prompting concerns among economists about
  the potential for reduced purchasing power and higher costs of living.
"""

classifier(text)

[{'label': 'negative', 'score': 0.5200349688529968}]

In [35]:
text = "The financial advisor recommended a balanced approach to investing, combining both growth and income strategies."

classifier = pipeline("sentiment-analysis", model = "jinxxx123/financial_sentiment_analysis")

classifier(text)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Device set to use cuda:0


[{'label': 'neutral', 'score': 0.9386593699455261}]

In [40]:
text = "Strong earnings reports drive the stock to new highs, boosting investor confidence."

classifier(text)

[{'label': 'positive', 'score': 0.9928140044212341}]